In [1]:
# !curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
# !unzip -q kagglecatsanddogs_3367a.zip
# !ls PetImages

In [2]:
import re
from glob import glob

import numpy as np
from scipy import stats
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
def rotate(x: tf.Tensor) -> tf.Tensor:
    # Rotate 0, 90, 180, 270 degrees
    return tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

def flip(x: tf.Tensor) -> tf.Tensor:
    x = tf.image.random_flip_left_right(x)
    return x

def color(x: tf.Tensor) -> tf.Tensor:
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

def augument(x):
    return flip(x)

In [4]:
image_size = (180, 180)

def get_val_ds(image_size, augument=False, augmentations=[flip, rotate]):

    batch_size = 32

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "PetImages",
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    
    if not augument:
        return val_ds

    for f in augmentations:
        val_ds = val_ds.map(lambda x, y: (f(x), y))
    val_ds = val_ds.map(lambda x, y: (tf.clip_by_value(x, 0, 1), y), num_parallel_calls=4)

    return val_ds

### model_re = re.compile('baseline_(\d+).h5')
models = sorted(glob('models/baseline_*.h5'), key=lambda x: int(model_re.findall(x)[0]))

In [6]:
model = keras.models.load_model(models[-1])

In [7]:
label = list()
predict = None
_tta_predict = None

for i, j in get_val_ds(image_size):

    if predict is None:
        predict = model.predict(i)
        _tta_predict = np.concatenate(list(map(model.predict, [i]+[augument(i) for _ in range(10)])), axis=1)
    else:
        predict = np.concatenate([predict, model.predict(i)])
        _tta_predict = np.concatenate([_tta_predict, np.concatenate(list(map(model.predict, [i]+[augument(i) for _ in range(10)])), axis=1)])
    label.extend(list(map(int, j)))

Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


## Baseline

In [8]:
accuracy_score(list(map(lambda x:x>0.5, predict.reshape(-1))), label)

0.9348568987612131

## Take the Mode as ensemble

In [9]:
se_predict = np.array(stats.mode(_tta_predict>0.5, axis=1))[0, :, :]
accuracy_score(se_predict, label)

0.9348568987612131

## Take the Mean as ensemble

In [10]:
se_predict = np.mean(_tta_predict, axis=1)>0.5
accuracy_score(se_predict, label)

0.9423323366082871